[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io)

# Machine Learning Methods

## Exercise 007 - Dimensionality Reduction

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.001 | 07/11/2025 | Royi Avital | Defined an objective                                               |
| 0.1.000 | 03/03/2023 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/MachineLearningMethods/2023_01/Exercise0007DimensionalityReduction.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.manifold import Isomap, MDS, SpectralEmbedding, TSNE
from sklearn.model_selection import cross_val_predict
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

# Miscellaneous
import gdown
import os
from platform import python_version
import random

# Typing
from typing import Callable, List, Tuple
from numpy.typing import NDArray

# Visualization
import matplotlib.pyplot as plt

# Jupyter
from IPython import get_ipython

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

In [ ]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2

DATA_FILE_URL   = r'https://drive.google.com/uc?export=download&confirm=9iBg&id=1UXLdZgXwClgwZVszRq88UaaN2nvgMFiC'
DATA_FILE_NAME  = r'BciData.npz'

In [ ]:
# Fixel Algorithms Packages


## Exercise

In this exercise we'll use _Dimensionality Reduction_ as a feature engineering process.  
We'll try 3 different approaches utilize the process: Linear, Non Linear and with manual feature engineering.

In this exercise:

1. We'll process real world EEG signals to identify movement of body parts.
2. We'll use dimensionality reduction with a classifier to identify the part of the body.
3. We'll optimize the combination of the dimensionality reduction and the classifier.
4. Visualize the features in low dimension (2).

* <font color='brown'>(**#**)</font> This exercise won't have single line completions. It will require coding the whole block according to instructions.

In [ ]:
# Parameters

numRowsPlot = 3
numColsPlot = 2

In [ ]:
# Auxiliary Functions

def PlotLabelsHistogram(vY: NDArray, hA = None, lClass = None, xLabelRot: int = None) -> plt.Axes:

    if hA is None:
        hF, hA = plt.subplots(figsize = (8, 6))
    
    vLabels, vCounts = np.unique(vY, return_counts = True)

    hA.bar(vLabels, vCounts, width = 0.9, align = 'center')
    hA.set_title('Histogram of Classes / Labels')
    hA.set_xlabel('Class')
    hA.set_ylabel('Number of Samples')
    hA.set_xticks(vLabels)
    if lClass is not None:
        hA.set_xticklabels(lClass)
    
    if xLabelRot is not None:
        for xLabel in hA.get_xticklabels():
            xLabel.set_rotation(xLabelRot)

    return hA

## Generate / Load Data

In this exercise we'll use the Brain Computer Interfaces (BCI) Data from [BCI Competition IV](https://www.bbci.de/competition/iv/).  
Specifically we'll use [data set 2a](https://www.bbci.de/competition/iv/#dataset2a) provided by the Institute for Knowledge Discovery (Laboratory of Brain-Computer Interfaces), Graz University of Technology, (Clemens Brunner, Robert Leeb, Gernot Müller-Putz, Alois Schlögl, Gert Pfurtscheller).  
This is a recording of EEG signals while the subject is doing a cued movement of the left hand, right hand, feet or tongue.

The data is composed of:

 * 22 EEG channels (0.5-100Hz; notch filtered).
 * 4 classes: left hand, right hand, feet, tongue.
 * 9 subjects.


In [ ]:
# Download Data
# This section downloads data from the given URL if needed.
# If it fails, download the data manually.

if not os.path.exists(DATA_FILE_NAME):
    gdown.download(DATA_FILE_URL, DATA_FILE_NAME)

In [ ]:
# Loading / Generating Data

dData = np.load(DATA_FILE_NAME)
mX    = dData['mX1']
vY    = dData['vY1']

# Sample: An observation of a subject.
# Measurement: Sample in time of the EEG signal.
# Channel: EEG Channel.
numSamples, numMeasurements, numChannels = mX.shape 
lLabel = ['Left Hand', 'Right Hand', 'Foot', 'Tongue'] #<! The labels

print(f'The data shape: {mX.shape}')

### Pre Process Data

Scale the pixels into the [0, 1] range.

In [ ]:
# Pre Process Data


### Plot the Data

In [ ]:
# Plot the Image

numPlots = numRowsPlot * numColsPlot

hF, hAs = plt.subplots(nrows = numRowsPlot, ncols = numColsPlot, figsize = (18, 10))
hAs = hAs.flat

vIdx = np.random.choice(numSamples, numPlots, replace = False)

for sampleIdx, hA in zip(vIdx, hAs):
    mXX = mX[sampleIdx, :, :]
    hA.plot(mXX)
    hA.set_title(f'EEG Signals of Sample {sampleIdx:04d} with Label {lLabel[vY[sampleIdx]]}')
    hA.set_xlabel('Measurement Index')
    hA.set_ylabel('Measurement Value')

hF.tight_layout()

In [ ]:
# Histogram of Labels

hA = PlotLabelsHistogram(vY, lClass = lLabel)

* <font color='red'>(**?**)</font> What score would you use in the case above?

## Model 001

In this model we'll use a linear combination of the features with a classifier.  
The cross validation is _leave one out_ using `cross_val_predict()`.

1. Transform data into `(numSamples, numMeasurements x numChannels)`.
2. Apply PCA with the optimal number of components.
3. Use a **non ensemble** classifier of your choice.

**Objective**: Above 35% accuracy in _Leave One Out_ cross validation.

* <font color='red'>(**?**)</font> What's the maximum number of components of the PCA you may use?

In [ ]:
?????

## Model 002

In this model we'll use the covariance matrix of the data as a feature.  
Basically we want to take advantage of the interaction between the different channels.

Do the following steps:

1. For each sample $\boldsymbol{X}_{i}\in\mathbb{R}^{1000 \times 22}$, compute the covariance matrix $\boldsymbol{C}_{i}\in\mathbb{R}^{22\times22}$.  
   You may use [`np.cov()`](https://numpy.org/doc/stable/reference/generated/numpy.cov.html).
2. Set $\boldsymbol{c}_{i}\in\mathbb{R}^{22^{2}}$ as the columns stack version of each $\boldsymbol{C}_{i}\in\mathbb{R}^{22 \times 22}$.  
   You may use [`np.reshape()`](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html).
4. Build the feature matrix based on $\boldsymbol{c}_{i}$.
5. Analyze the separation using a non linear dimensionality reduction model (Use `vY` for the colorization of data).
6. Use an ensemble based classifier.

You should optimize the combination of the classification model and the dimensionality reduction model.

**Objective**: Above 50% accuracy in _Leave One Out_ cross validation.

* <font color='brown'>(**#**)</font> You should also try no dimensionality reduction as well.
* <font color='brown'>(**#**)</font> If you use `t-SNE`, out of consideration for _run time_, you may want to apply `PCA` before in order to reduce the data into ~50 dimensions.

In [ ]:
?????

## Model 003

In this model we'll use the covariance matrix of the data as a feature.  
Yet, we'll add a covariance matrix specific metric, the SPD Metric (`SpdMetric`):

$$d\left(\boldsymbol{C}_{i},\boldsymbol{C}_{j}\right)=\sqrt{\sum_{i=1}^{d}\log^{2}\left(\lambda_{i}\left(\boldsymbol{C}_{i}^{-1}\boldsymbol{C}_{j}\right)\right)}$$

Where ${\lambda}_{i} \left( \cdot \right)$ extract the $i$ -th eigen value of the matrix.

We'll use this metric to apply a metric aware dimensionality reduction.

Do the following steps:

1. Implement the `SpdMetric` metric.  
   The function `SpdMetric()` input is 2 column stacked covariance matrix $\boldsymbol{c}_{i}$ and $\boldsymbol{c}_{j}$.  
   The function reshapes them back to the two matrices $\boldsymbol{C}_{i}$ and $\boldsymbol{C}_{j}$ and returns $d\left(\boldsymbol{C}_{i},\boldsymbol{C}_{j}\right)$.
2. Based covariance features from _Model 002_ build a distance matrix `mD` based on the metric above.  
   Namely `mD[ii, jj]` should be the distance between the covariance matrix of the `ii` sample to the `jj` sample.
5. Analyze the separation using a non linear dimensionality reduction model (Use `vY` for the colorization of data).  
   Make sure to utilize the distance function above.
6. Use an ensemble based classifier applied on the dimensionality reduced data.

You should optimize the combination of the classification model and the dimensionality reduction model.

* <font color='brown'>(**#**)</font> You may use `sp.linalg.eigvalsh(mCi, mCj)` to calculate $ \lambda_{i}\left(\boldsymbol{C}_{i}^{-1}\boldsymbol{C}_{j}\right) $.

**Objective**: Above 75% accuracy in _Leave One Out_ cross validation.

In [ ]:
?????